In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Multiple_sclerosis/GSE171950'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Microarray based prediction of circular RNA expression in multiple sclerosis"
!Series_summary	"The aim was to investigate the total circular RNA profile in patients with relapsing–remitting multiple sclerosis and healthy control."
!Series_summary	"We analyzed close to 14,000 individual circRNA per sample."
!Series_overall_design	"Hybridization microarray was used to define the circRNA profile in peripheral blood mononuclear cells (PBMC) from 20 untreated patients with relapsing-remitting multiple sclerosis (10 in relapse, 10 in remission) and 10 healthy controls."
Sample Characteristics Dictionary:
{0: ['disease state: multiple sclerosis relapse', 'disease state: multiple sclerosis remission', 'disease state: healthy control'], 1: ['cell type: peripheral blood mononuclear cells (PBMC)']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset likely contains gene expression data
is_gene_available = True  # Based on the description, the dataset seems to focus on circRNA, which are often analyzed through gene expression microarrays

# Identify the availability of variables in the sample characteristics dictionary
trait_row = 0  # disease state information is available under key 0
# Age and gender information are not explicitly available from the provided dictionary
age_row = None
gender_row = None

# Define the conversion functions
def convert_trait(value):
    """
    Convert the disease state information to binary values.
    healthy control -> 0, multiple sclerosis (relapse/remission) -> 1
    """
    value = value.split(':')[1].strip()
    if value == 'healthy control':
        return 0
    elif value in ['multiple sclerosis relapse', 'multiple sclerosis remission']:
        return 1
    else:
        return None

# Age and gender data are not available in this dataset
convert_age = lambda value: None
convert_gender = lambda value: None

# Save cohort information
save_cohort_info('GSE171950', './preprocessed/Multiple_sclerosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction when trait_row is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Multiple_sclerosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Multiple_sclerosis/trait_data/GSE171950.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM5238634': [1], 'GSM5238635': [1], 'GSM5238636': [1], 'GSM5238637': [1], 'GSM5238638': [1], 'GSM5238639': [1], 'GSM5238640': [1], 'GSM5238641': [1], 'GSM5238642': [1], 'GSM5238643': [1], 'GSM5238644': [1], 'GSM5238645': [1], 'GSM5238646': [1], 'GSM5238647': [1], 'GSM5238648': [1], 'GSM5238649': [1], 'GSM5238650': [1], 'GSM5238651': [1], 'GSM5238652': [1], 'GSM5238653': [1], 'GSM5238654': [0], 'GSM5238655': [0], 'GSM5238656': [0], 'GSM5238657': [0], 'GSM5238658': [0], 'GSM5238659': [0], 'GSM5238660': [0], 'GSM5238661': [0], 'GSM5238662': [0], 'GSM5238663': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ASCRP3000001', 'ASCRP3000002', 'ASCRP3000003', 'ASCRP3000004',
       'ASCRP3000005', 'ASCRP3000006', 'ASCRP3000007', 'ASCRP3000008',
       'ASCRP3000009', 'ASCRP3000010', 'ASCRP3000011', 'ASCRP3000012',
       'ASCRP3000013', 'ASCRP3000014', 'ASCRP3000015', 'ASCRP3000016',
       'ASCRP3000017', 'ASCRP3000018', 'ASCRP3000019', 'ASCRP3000020'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ASCRP3000001', 'ASCRP3000002', 'ASCRP3000003', 'ASCRP3000004', 'ASCRP3000005'], 'circRNA': ['hsa_circRNA_082501', 'hsa_circRNA_407247', 'hsa_circRNA_007418', 'hsa_circRNA_407121', 'hsa_circRNA_407087'], 'TYPE': ['circRNA', 'circRNA', 'circRNA', 'circRNA', 'circRNA'], 'BUILD': ['HG19', 'HG19', 'HG19', 'HG19', 'HG19'], 'SEQUENCE': ['AAAAAAACAACCAAAAAATGTTCAACAGCATGAGAAGGTTCAGAAAGCCAGTACAGAGGG', 'AAAAAACGAAGAAAAAGAGACACCCAGCTCACCTCCAAGTTTGCCTGCAGGAGCCGGCTC', 'AAAAAACTAGAAATGTGTTCAGAAATTAAAGGTCCACAGAAGGAGGGCCTGTCCTCCCAA', 'AAAAAAGAAGTCCTCGGATACGCTGAAACTGCAGAAGAAAGCAAAAAAAGGTGGGACCAG', 'AAAAAAGGAACAGGAAAACTACATCTCAGACCCAACTATCTTATGGAGGGCTTATCCCCA']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the identifier and gene symbol keys from the gene annotation data preview.
identifier_key = 'ID'
gene_symbol_key = 'circRNA'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Multiple_sclerosis/gene_data/GSE171950.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Multiple_sclerosis')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE171950', './preprocessed/Multiple_sclerosis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Multiple_sclerosis/GSE171950.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
